In [8]:
import numpy as np
import math
from scipy.stats import norm

## 1 Poisson process blocking system

In [17]:
def simulate_blocking_system(m=10, mean_service=8, mean_arrival=1, total_customers=10_000):

    # in Poisson process interarrival times are exponentially distributed
    between_arrival_times = np.random.exponential(mean_arrival, size=total_customers)
    # cumulative sum to get wall clock arrival time
    arrival_times = np.cumsum(between_arrival_times)

    # exponential service time
    service_times = np.random.exponential(mean_service, size=total_customers)

    # simulation
    server_end_times = np.zeros(m)
    blocked = 0

    for arrival, service in zip(arrival_times, service_times):
        # Find first free server
        free_servers = np.where(server_end_times <= arrival)[0]

        if len(free_servers) > 0:
            chosen = free_servers[0]
            server_end_times[chosen] = arrival + service
        else:
            blocked += 1

    blocked_fraction = blocked / total_customers

    return blocked_fraction


In [19]:
m = 10 #service units
mean_arrival = 1
mean_service = 8
total_customers = 10_000
A = mean_arrival * mean_service

In [31]:
blocked_fractions = []
for _ in range(10):
    blocked_frac = simulate_blocking_system()
    blocked_fractions.append(blocked_frac)
print(blocked_fractions)

def blocked_ci(blocked_fractions):
    blocked_fraction = np.mean(blocked_fractions)
    print(f"Average blocked fraction: {blocked_fraction:.4f}")
    z = norm.ppf(0.975)
    se = np.sqrt(blocked_fraction * (1 - blocked_fraction) / total_customers)
    ci_lower = blocked_fraction - z * se
    ci_upper = blocked_fraction + z * se
    print(f"95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")

blocked_ci(blocked_fractions)

[0.1232, 0.1116, 0.1119, 0.1088, 0.1234, 0.1138, 0.1331, 0.1186, 0.1318, 0.1272]
Average blocked fraction: 0.1203
95% CI: [0.1140, 0.1267]


Exact solution:

In [30]:
s = 0
for i in range(m):
    s += (A**i) / math.factorial(i)
(A**m/math.factorial(m))/s

0.13851266214160918

## 2 Renewal process with different arrival time distributions

In [35]:
def renewal_system(arrival_times, m=10, mean_service=8, total_customers=10_000):

    # exponential service time
    service_times = np.random.exponential(mean_service, size=total_customers)

    # simulation
    server_end_times = np.zeros(m)
    blocked = 0

    for arrival, service in zip(arrival_times, service_times):
        # Find first free server
        free_servers = np.where(server_end_times <= arrival)[0]

        if len(free_servers) > 0:
            chosen = free_servers[0]
            server_end_times[chosen] = arrival + service
        else:
            blocked += 1

    blocked_fraction = blocked / total_customers

    return blocked_fraction


### Erlang arrival times

In [33]:
def erlang_arrival_times(rate, k, n):
    inter_arrival_times = np.random.gamma(shape=k, scale=1/rate, size=n)
    arrival_times = np.cumsum(inter_arrival_times)
    return arrival_times.tolist(), inter_arrival_times.tolist()

In [42]:
rate = 1  # λ = 1 (mean = 1)
k = 3    # shape parameter (Erlang)
n = total_customers    
arrival_times, inter_arrivals = erlang_arrival_times(rate, k, n)

print("Inter-arrival times:", inter_arrivals)
print("Arrival times:", arrival_times)

Inter-arrival times: [2.624469720984314, 2.0110991449598297, 3.2936854020476543, 4.359535066988473, 2.7305684130969894, 1.9090119062895927, 5.697052948211746, 1.07311117721206, 6.6583756678580714, 3.286262611321769, 0.8333450027403032, 6.490165777227348, 2.0272526894940297, 4.598859154769903, 2.0683747518358677, 2.776421121427311, 4.9655974529738, 5.598213919219571, 7.1907183770084835, 3.3062243804476084, 3.8291328886772416, 1.9264721095191644, 4.35256586340555, 5.423359308605539, 2.996509824033915, 2.798647221646468, 2.524613205398173, 2.4523600009201156, 1.947932957194379, 2.3240383568716725, 3.914336533866498, 4.026699613036935, 3.2673539624625474, 1.1774990254290958, 0.7370555302359953, 6.832643557817686, 2.738741370721945, 3.1172126201324155, 4.89612463358872, 6.185901996416529, 4.658849911358657, 3.2532186895749646, 2.139571674320087, 3.036907640696329, 4.702545690863963, 2.3817440805991295, 1.3934143902075617, 3.7632571129578825, 6.542596371582585, 3.6226753941442973, 2.97339958

In [50]:
ks = []
for k in [0.1,0.5,1,2]:
    blocked_fractions = []
    for _ in range(10):
        arrival_times, inter_arrivals = erlang_arrival_times(rate, k, n)
        blocked_frac = renewal_system(arrival_times)
        blocked_fractions.append(blocked_frac)
    print(f"k = {k}:")
    blocked_ci(blocked_fractions)

k = 0.1:
Average blocked fraction: 0.8804
95% CI: [0.8740, 0.8868]
k = 0.5:
Average blocked fraction: 0.4650
95% CI: [0.4552, 0.4747]
k = 1:
Average blocked fraction: 0.1242
95% CI: [0.1177, 0.1307]
k = 2:
Average blocked fraction: 0.0010
95% CI: [0.0004, 0.0016]


### Hyper-exponential arrival times

In [ ]:
def hyper_exponential_arrival_times(lambdas, probs, n):
    chosen_indices = np.random.choice(len(lambdas), size=n, p=probs)
    inter_arrival_times = [np.random.exponential(1 / lambdas[i]) for i in chosen_indices]
    arrival_times = np.cumsum(inter_arrival_times)
    return arrival_times.tolist(), inter_arrival_times

In [52]:
lambdas = [0.8333, 5.0]
probs = [0.8, 0.2]

for _ in range(10):
    arrival_times, inter_arrivals = hyper_exponential_arrival_times(lambdas, probs, n)
    blocked_frac = renewal_system(arrival_times)
    blocked_fractions.append(blocked_frac)
blocked_ci(blocked_fractions)

Average blocked fraction: 0.0681
95% CI: [0.0632, 0.0730]


## 3 Poisson process with different service time distributions

In [53]:
def poisson_process(service_times, m=10, mean_arrival=1, total_customers=10_000):

    # in Poisson process interarrival times are exponentially distributed
    between_arrival_times = np.random.exponential(mean_arrival, size=total_customers)
    # cumulative sum to get wall clock arrival time
    arrival_times = np.cumsum(between_arrival_times)

    # simulation
    server_end_times = np.zeros(m)
    blocked = 0

    for arrival, service in zip(arrival_times, service_times):
        # Find first free server
        free_servers = np.where(server_end_times <= arrival)[0]

        if len(free_servers) > 0:
            chosen = free_servers[0]
            server_end_times[chosen] = arrival + service
        else:
            blocked += 1

    blocked_fraction = blocked / total_customers

    return blocked_fraction

### Constant service time

In [56]:
# constant should be same as mean = 8
for _ in range(10):
    service_times = np.ones(n) * mean_service
    blocked_frac = poisson_process(service_times)
    blocked_fractions.append(blocked_frac)
blocked_ci(blocked_fractions)

Average blocked fraction: 0.1083
95% CI: [0.1022, 0.1144]


### Pareto service times

In [57]:
def pareto_service_times(k, scale, n):
    # NumPy's Pareto returns values for (x/x_m - 1)
    samples = (np.random.pareto(k, size=n) + 1) * scale
    return samples.tolist()

In [58]:
for k in [1.05,2.05]:
    for _ in range(10):
        service_times = pareto_service_times(k,1,n)
        blocked_frac = poisson_process(service_times)
        blocked_fractions.append(blocked_frac)
    print(f"k = {k}")
    blocked_ci(blocked_fractions)

k = 1.05
Average blocked fraction: 0.1092
95% CI: [0.1031, 0.1153]
k = 2.05
Average blocked fraction: 0.0936
95% CI: [0.0879, 0.0993]


### Weibull service times

In [64]:
def weibull_service_times(k, n):
    samples = np.random.weibull(k, size=n)
    return samples.tolist()

In [72]:
for k in [0.5,1,1.5, 5]:
    for _ in range(10):
        service_times = weibull_service_times(k,n)
        blocked_frac = poisson_process(service_times)
        blocked_fractions.append(blocked_frac)
    print(f"k = {k}")
    blocked_ci(blocked_fractions)

k = 0.5
Average blocked fraction: 0.0126
95% CI: [0.0104, 0.0148]
k = 1
Average blocked fraction: 0.0124
95% CI: [0.0102, 0.0145]
k = 1.5
Average blocked fraction: 0.0121
95% CI: [0.0100, 0.0143]
k = 5
Average blocked fraction: 0.0119
95% CI: [0.0098, 0.0140]
